In [80]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

from tqdm import tqdm
from src.scraper import main

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
main()

In [39]:
url = "https://ph.indeed.com/jobs?q=data+analyst%2C+data+science%2C+data+engineer&l=quezon+city&vjk=5d9e69f96362cff0"


headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    }
response = requests.get(url=url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
    _ids = list()
    a_elements = soup.findAll('a')
    
    for element in a_elements:
        element_id = element.get("id")
        if element_id:
            _ids.append(element_id)
            
    _ids = [id for id in _ids if id.startswith("sj_")]

    
    # for location in locations:
    #     extract_locations.append(location.span.text)

In [46]:
_ids

['sj_1199a7ea40de655c',
 'sj_f9947544b305fff6',
 'sj_5f7d662da7e9b9d2',
 'sj_ff99cf1fbe07d3ef',
 'sj_f479e012d42b484f',
 'sj_25a02547808d6d2b',
 'sj_3db19cbe47f43233',
 'sj_136bdc06cfa8a724',
 'sj_238bdf1694335d83',
 'sj_c87f6edd0b229722',
 'sj_2107726ea60c8b8b',
 'sj_1fec64ac69f21f7b',
 'sj_58a215b7191b3480']

In [89]:
input_keywords = urllib.parse.quote(input("Enter skill/s: "))
input_location = urllib.parse.quote(input("Enter location: "))

base_url = f"https://ph.indeed.com/jobs?q={input_keywords}&l={input_location}"

ids = []
titles = []
companies = []
locations = []
descriptions = []

for idx in range(0, 200, 10):
    # print(idx)
    curr_url = f"{base_url}&start={idx}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    }

    response = requests.get(url=base_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        a_elements = soup.find_all('a')
        _ids = [element.get("id") for element in a_elements if element.get("id") and element.get("id").startswith("sj_")]



        for _id in tqdm(_ids):
            url = f"{base_url}&vjk={_id.split('_')[1]}"
            response = requests.get(url=url, headers=headers)

            if response.status_code == 200:
                soup_inner = BeautifulSoup(response.content, "html.parser")

                job_titles = soup_inner.find_all("h2", attrs={"data-testid": "jobsearch-JobInfoHeader-title"})
                job_company = soup_inner.find_all("a", class_="css-1ioi40n e19afand0")
                job_locations = soup_inner.find_all("div", id="jobLocationText")
                job_description = soup_inner.find_all("div", id="jobDescriptionText")

                # Extracting and storing titles, companies, and locations
                for title, company, location in zip(job_titles, job_company, job_locations):
                    ids.append(_id)
                    titles.append(title.text.strip())
                    companies.append(company.text.strip())
                    locations.append(location.text.strip())
                    descriptions.append(job_description.text.strip())

 40%|████      | 2/5 [00:01<00:02,  1.02it/s]


AssertionError: URL returned successfully

css-1ioi40n e19afand0

In [90]:
jobs_listing = pd.DataFrame({
    "id": ids,
    "title": titles,
    "company": companies,
    "location": locations
})

jobs_listing

,id,title,company,location
0,sj_c87f6edd0b229722,Reports Analyst - job post,Enshored,Pasig
1,sj_f0e0fec1c25e6a67,Level 2 Security Analyst - job post,Buchanan Technologies,Manila
2,sj_d3d3e724a1b93669,Loan Servicing Analyst - job post,JPMorgan Chase & Co,Manila
3,sj_c87f6edd0b229722,Reports Analyst - job post,Enshored,Pasig
4,sj_67fe7b593107f8fe,Reporting Senior Analyst - job post,Accenture Inc.,Manila
5,sj_f5ddb419b491153c,3PL Management Analyst - job post,FAST LOGISTICS,Parañaque
6,sj_c87f6edd0b229722,Reports Analyst - job post,Enshored,Pasig
7,sj_67fe7b593107f8fe,Reporting Senior Analyst - job post,Accenture Inc.,Manila
8,sj_c87f6edd0b229722,Reports Analyst - job post,Enshored,Pasig
9,sj_a3190dadf299d2e2,Market Research Analyst - job post,Casa Bella Home and Living Corp,Pasig
